In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score

sns.set(style="whitegrid")

# Load data
df = pd.read_csv("data_cleaned.csv")

# Feature selection
cluster_features = [
    "acousticness", "danceability", "duration_ms", "energy",
    "instrumentalness", "liveness", "loudness", "speechiness",
    "tempo", "valence"
]
cluster_features = [c for c in cluster_features if c in df.columns]
data = df.dropna(subset=cluster_features).copy()

print("Total samples:", len(data))
print("Features:", cluster_features)

# Sample for faster k search
max_sample = 20000
data_sample = data.sample(min(max_sample, len(data)), random_state=42)

scaler = StandardScaler()
X_sample_scaled = scaler.fit_transform(data_sample[cluster_features])

# Smaller subset for silhouette
sil_max = 5000
idx_sil = np.random.choice(
    X_sample_scaled.shape[0],
    size=min(sil_max, X_sample_scaled.shape[0]),
    replace=False
)
X_sil = X_sample_scaled[idx_sil]

# Try k values
k_values = range(2, 11)
inertias = []
sil_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_sample = kmeans.fit_predict(X_sample_scaled)
    inertias.append(kmeans.inertia_)
    labels_sil = kmeans.predict(X_sil)
    sil_scores.append(silhouette_score(X_sil, labels_sil))

plt.figure(figsize=(8, 4))
plt.plot(k_values, inertias, marker="o")
plt.title("Elbow")
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(k_values, sil_scores, marker="o")
plt.title("Silhouette")
plt.tight_layout()
plt.show()

best_k = k_values[int(np.argmax(sil_scores))]
print("Best k:", best_k)

# Final model
X_full_scaled = scaler.transform(data[cluster_features])
kmeans_final = KMeans(n_clusters=best_k, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_full_scaled)
data["cluster"] = cluster_labels

# Quick silhouette and DB index
full_sil_max = 5000
idx_full = np.random.choice(
    X_full_scaled.shape[0],
    size=min(full_sil_max, X_full_scaled.shape[0]),
    replace=False
)
sil_final = silhouette_score(X_full_scaled[idx_full], cluster_labels[idx_full])
db_final = davies_bouldin_score(X_full_scaled, cluster_labels)

print("Silhouette:", sil_final)
print("Davies Bouldin:", db_final)

print(data["cluster"].value_counts().sort_index())

# Cluster profile
cluster_profile = data.groupby("cluster")[cluster_features].mean()
print(cluster_profile)

plt.figure(figsize=(10, 6))
sns.heatmap(cluster_profile, annot=True, fmt=".2f", cmap="coolwarm")
plt.tight_layout()
plt.show()

# Popularity per cluster
if "popularity" in data.columns:
    pop_by_cluster = data.groupby("cluster")["popularity"].mean()
    print(pop_by_cluster)
    pop_by_cluster.plot(kind="bar")
    plt.title("Popularity by cluster")
    plt.tight_layout()
    plt.show()

# Genre distribution
if "main_genre" in data.columns:
    for c in sorted(data["cluster"].unique()):
        print(f"\nCluster {c}:")
        print(data[data["cluster"] == c]["main_genre"].value_counts().head(10))

# PCA plot
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_full_scaled)
data["pca1"] = X_pca[:, 0]
data["pca2"] = X_pca[:, 1]

plot_sample = data.sample(min(8000, len(data)), random_state=42)

plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=plot_sample,
    x="pca1", y="pca2",
    hue="cluster", palette="tab10", alpha=0.5
)
plt.title("PCA clusters")
plt.tight_layout()
plt.show()

# Quick ranking helper
print("\nCluster rankings:")
key_feats = ["danceability", "energy", "acousticness", "instrumentalness", "valence", "tempo"]
key_feats = [c for c in key_feats if c in cluster_profile.columns]

for feat in key_feats:
    print(f"\n{feat}:")
    print(cluster_profile[feat].sort_values(ascending=False))
